In [1]:
from data.process import *

2024-07-17 21:11:06.907733: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 21:11:06.915949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 21:11:06.926269: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 21:11:06.926295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 21:11:06.933903: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_dataframe()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu
import numpy as np
from model.catboost_opt import *
import optuna
import pprint

In [4]:
dataset = cb.Pool(X_train, Y_train, cat_features=np.where(X_train.dtypes != np.float64)[0])

In [5]:
tuned_params = {
    **const_params,
    'learning_rate': 0.01,
    'depth': 10,
    'bagging_temperature': 0.8,
    'l2_leaf_reg': 0.005,
    'random_strength': 0,
    'border_count': 32,
    'iterations': 5000,
    'early_stopping_rounds': 200,
}

In [6]:
model = cb.CatBoostClassifier(**tuned_params, cat_features=np.where(X_train.dtypes != np.float64)[0])
model.fit(dataset, verbose=False, eval_set=(X_dev, Y_dev), plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because AUC is/are not implemented for GPU


: 

In [ ]:
model.save_model('catboost_model.json', format='json', pool=dataset)